# Multivariate polynomials implementations

**Contributed by**: Benoît Legat

The SumOfSquares package is built on top of the [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl)
abstract interface. [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
is an implementation of this abstract interface so it can be used with
SumOfSquares. Moreover, any other implementation can be used as well. To
illustrate, we solve Examples 3.38 of [BPT12] with
[TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl),
another implementation of [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl).

[BPT12] Blekherman, G. & Parrilo, P. A. & Thomas, R. R.
*Semidefinite Optimization and Convex Algebraic Geometry*.
Society for Industrial and Applied Mathematics, **2012**.

In [1]:
import TypedPolynomials
TypedPolynomials.@polyvar x y
using SumOfSquares
import CSDP
model = SOSModel(CSDP.Optimizer)
con_ref = @constraint(model, 2x^4 + 5y^4 - x^2*y^2 >= -2(x^3*y + x + 1))
optimize!(model)
solution_summary(model)

Iter: 10 Ap: 9.68e-01 Pobj: -1.0000000e+00 Ad: 9.68e-01 Dobj: -1.0000000e+00 
Success: SDP solved
Primal objective value: -1.0000000e+00 
Dual objective value: -1.0000000e+00 
Relative primal infeasibility: 2.08e-15 
Relative dual infeasibility: 7.87e-10 
Real Relative Gap: -6.56e-10 
XZ Relative Gap: 1.03e-09 
DIMACS error measures: 3.97e-15 0.00e+00 1.87e-09 0.00e+00 -6.56e-10 1.03e-09
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.39e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  9.2771541e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.3401544e+01 


* Solver : CSDP

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution
  Objective value      : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 4.28979e-02


We see that the problem is feasible. The Sum-of-Squares decomposition can be
obtained as follows:

In [2]:
sos_decomposition(con_ref)

(-0.8025918219908258*x^2 - 0.12696384429349364*x*y + 2.193153675722526*y^2 - 0.4175969723687504*x - 0.09754557420421368*y - 0.6797572045423704)^2 + (-0.9300194624998961*x^2 - 1.2293223495653895*x*y - 0.107702537307966*y^2 + 0.2742242305063454*x - 0.6454996724366074*y + 0.9043636674517767)^2 + (0.44438734692024634*x^2 - 0.1518231994578531*x*y - 0.2040923091350899*y^2 - 0.6219741211964985*x - 1.3853658247098444*y - 0.5739116466586851)^2 + (0.02094358532346286*x^2 - 0.7618740018659129*x*y - 0.2555836507088663*y^2 - 0.34369066581037083*x + 0.5184207196796033*y - 0.570289485543629)^2 + (-0.5053882307722652*x^2 + 0.2972258345292787*x*y - 0.24790977216150892*y^2 - 0.4999834602481285*x + 0.0491342550895953*y + 0.04145310741757451)^2 + (-0.19384054006505613*x^2 + 0.05960947731636697*x*y - 0.10020573751666935*y^2 + 0.2539007344344329*x - 0.06335659197202279*y - 0.2524548756891834)^2

Why is there several implementations ?
Depending in the use-case, one implementation may be more appropriate than
another one. [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is faster than [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
but it requires new compilation whenever the list of variables changes.
This means that [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is not appropriate when the number of variables is dynamic or too large.
However, for a small number of variables, it can be faster.
When solving Sum-of-Squares programs, the time is mostly taken by the Semidefinite programming solver.
The time taken by SumOfSquares/JuMP/MathOptInterface are usually negligible
or it time is taken by manipulation of JuMP or MathOptInterface functions
therefore using TypedPolynomials over DynamicPolynomials may not make much difference in most cases.

One case for which using TypedPolynomials might be adequate is when
using domain defined by equalities (possibly also with inequalities).
Indeed, in that case, SumOfSquares computes the corresponding Gröbner basis which
may take a non-negligible amount of time for large systems of equalities.

To illustrate this, consider the computation of Gröbner basis for the
following system from [CLO05, p. 17].
The time taken by TypedPolynomials is below:

[CLO05] Cox, A. David & Little, John & O'Shea, Donal
*Using Algebraic Geometry*.
Graduate Texts in Mathematics, **2005**.
https://doi.org/10.1007/b138611

In [3]:
using BenchmarkTools
@btime let
    TypedPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.computegröbnerbasis!(S.I)
end

  244.503 μs (3305 allocations: 449.61 KiB)


true

The time taken by DynamicPolynomials is as follows:

In [4]:
import DynamicPolynomials
@btime let
    DynamicPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.computegröbnerbasis!(S.I)
end

  763.108 μs (6203 allocations: 574.66 KiB)


true

We see that TypedPolynomials is faster.
The time is still negligible for this small system but for larger systems, choosing TypedPolynomials may be helpful.
We can use this system in a Sum-of-Squares constraint as follows:

In [5]:
TypedPolynomials.@polyvar x y
S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
poly = -6x - 4y^3 + 2x*y^2 + 6x^3 - 3y^4 + 13x^2 * y^2
model = Model(CSDP.Optimizer)
con_ref = @constraint(model, poly in SOSCone(), domain = S)
optimize!(model)
solution_summary(model)

Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 1.97e-16 
Relative dual infeasibility: 5.00e-11 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 3.04e-10 
DIMACS error measures: 2.46e-16 0.00e+00 5.00e-11 0.00e+00 0.00e+00 3.04e-10
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.64e-01 Pobj:  0.0000000e+00 Ad: 8.07e-01 Dobj:  2.6104524e+00 
Iter:  2 Ap: 7.90e-01 Pobj:  0.0000000e+00 Ad: 7.40e-01 Dobj: -5.9858991e-02 
Iter:  3 Ap: 7.78e-01 Pobj:  0.0000000e+00 Ad: 7.55e-01 Dobj:  5.6667126e-01 
Iter:  4 Ap: 7.59e-01 Pobj:  0.0000000e+00 Ad: 8.08e-01 Dobj:  4.9238805e-01 
Iter:  5 Ap: 7.15e-01 Pobj:  0.0000000e+00 Ad: 6.42e-01 Dobj:  4.3470535e-01 
Iter:  6 Ap: 6.45e-01 Pobj:  0.0000000e+00 Ad: 7.00e-01 Dobj:  2.1752352e-01 
Iter:  7 Ap: 6.50e-01 Pobj:  0.0000000e+00 Ad: 6.29e-01 Dobj:  1.2995658e-01 
Iter:  8 Ap: 

* Solver : CSDP

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution
  Objective value      : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 6.39430e-01


We obtain the following decomposition:

In [6]:
dec = sos_decomposition(con_ref, 1e-6)

(-19.455175815853472*x^2 + 21.754662829241905*x*y - 5.464314355434417*y^2 - 0.00036597435076766847*x + 0.0002666415523617743*y + 7.581243238666406e-5)^2 + (-16.72035526740944*x^2 - 10.610525738832798*x*y + 17.2883614371507*y^2 - 0.00046612893315998007*x - 0.00048049302899887803*y - 1.572847989145024e-5)^2 + (-0.07188392155263187*x^2 - 0.09664733160370975*x*y - 0.12883836375349522*y^2 - 0.004371412269782948*x + 0.0032120020714792906*y + 4.456100422298629e-5)^2 + (0.0007696846260251474*x^2 + 0.0010332353482765122*x*y + 0.001378235821338183*y^2 - 0.04902409715253756*x + 0.036877686904439134*y + 2.8952637949808214e-5)^2

We can verify that it is correct as follows:

In [7]:
rem(dec - poly, S.I)

-7.456169418917566e-6x² - 1.9850689602379616e-5xy - 1.3185022282119919e-5y² + 1.692071406579094e-7x + 2.1583931733999258e-7y + 8.81884832569194e-9

Note that the difference between `dec` and `poly` is larger
than between the full gram matrix because `dec` is obtained by dropping
the lowest eigenvalues with the threshold `1e-6`; see `sos_decomposition`.

In [8]:
rem(gram_matrix(con_ref) - poly, S.I)

0.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*